In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import os
import cv2
import time
import tensorflow as tf

from detector import Detector

In [ ]:
MODEL_PATH = 'model.pb'
detector = Detector(MODEL_PATH, gpu_memory_fraction=0.5, visible_device_list='0')

# Get an image

In [ ]:
path = '/home/dan/datasets/COCO/images/train2017/'
i = os.listdir(path)[0]
path = path + i
image_array = cv2.imread(path)
image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image_array)
image

# Show detections

In [ ]:
with open('../data/coco_labels.txt', 'r') as f:
    labels = {line.strip(): i for i, line in enumerate(f.readlines()) if line.strip()}
    
decoder = {i: n for n, i in labels.items()}

In [ ]:
def draw_boxes_on_image(image, boxes, labels, scores):

    image_copy = image.copy()
    draw = ImageDraw.Draw(image_copy, 'RGBA')
    width, height = image.size
    scaler = np.array([height, width, height, width], dtype='float32')
    boxes = boxes.copy() * scaler

    for b, l, s in zip(boxes, labels, scores):
        ymin, xmin, ymax, xmax = b
        fill = (255, 0, 0, 45)
        outline = 'red'
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )
        draw.text((xmin, ymin), text='{0} {1:.3f}'.format(decoder[l], s))
    return image_copy

In [ ]:
boxes, labels, scores = detector(image_array, score_threshold=0.6)
draw_boxes_on_image(Image.fromarray(image_array), boxes, labels, scores)

# Measure speed

In [ ]:
times = []
for _ in range(110):
    start = time.perf_counter()
    boxes, labels, scores = detector(image_array, score_threshold=0.25)
    times.append(time.perf_counter() - start)
    
times = np.array(times)
times = times[10:]
print(times.mean(), times.std())